# Import、載入資料

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from keras import layers
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import scale
from mlxtend.plotting import scatterplotmatrix

In [ ]:
def load_data(housing_csv):
  csv_path = os.path.join('/content/drive/MyDrive/MachineLearn/house-regression/',housing_csv)
  return pd.read_csv(csv_path)

In [ ]:
train_originData = load_data('train-v3.csv')
valid_originData = load_data('valid-v3.csv')
test_originData = load_data('test-v3.csv')

#圖像整理(資料分析)

In [ ]:
train_originData.head()

In [ ]:
plt.figure(figsize = (8,10))
train_originData.corr()["price"].sort_values().drop("price").plot(kind = "barh");

In [ ]:
plt.hist(train_originData['price'],bins=30)
plt.show()

In [ ]:
var = 'sqft_living'
data = pd.concat([train_originData['price'], train_originData[var]], axis=1)
data.plot.scatter(x=var, y='price',xlim=(0,25000), ylim=(0,8500000));

In [ ]:
var = 'zipcode'
data = pd.concat([train_originData['price'], train_originData[var]], axis=1)
data.plot.scatter(x=var, y='price',xlim=(97990,98210), ylim=(0,8000000));

In [ ]:
var = 'long'
data = pd.concat([train_originData['lat'], train_originData[var]], axis=1)
data.plot.scatter(x=var, y='lat',xlim=(-122.55,-121.28), ylim=(47.1,47.85),alpha = 0.1);

In [ ]:
sns.boxplot(x = 'sale_month', y = 'price', data = train_originData);

In [ ]:
train_originData.groupby('sale_month')['price'].mean().plot();

In [ ]:
#cols = ['sale_yr','sale_month','sale_day','bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated','zipcode','lat','long','sqft_living15','sqft_lot15','price']
#scatterplotmatrix(train_originData[cols].values.astype(float), figsize=(50,50), names=cols, alpha=0.5)
#plt.tight_layout()
#plt.show()

# 資料預處理

In [ ]:
train_Data = train_originData
valid_Data = valid_originData
test_Data = test_originData

In [ ]:
train_originData.head(5)

In [ ]:
train_Data.loc[train_Data['yr_renovated'] >0 ,'yr_built'] = train_Data['yr_renovated']
valid_Data.loc[valid_Data['yr_renovated'] >0 ,'yr_built'] = valid_Data['yr_renovated']
test_Data.loc[test_Data['yr_renovated'] >0 ,'yr_built'] = test_Data['yr_renovated']

In [ ]:
train_Data = train_Data[train_Data['bedrooms'] < 11]
train_Data = train_Data[train_Data['sqft_living'] < 12000]
train_Data = train_Data[train_Data['sqft_lot15'] < 800000]

In [ ]:
#dropData = {'id','sale_yr','sale_month','sale_day','zipcode','view','waterfront','yr_renovated'}
dropData = {'id'}

In [ ]:
train_Data = train_originData.drop(dropData,axis=1)
valid_Data = valid_originData.drop(dropData,axis=1)
test_Data = test_originData.drop(dropData,axis=1)

In [ ]:
x_train = train_Data.drop(['price'],axis=1)
y_train = train_Data['price'].values
x_valid = valid_Data.drop(['price'],axis=1)
y_valid = valid_Data['price'].values
x_test = test_Data

In [ ]:
train_Data.corr()["price"].sort_values(ascending=False)

In [ ]:
#mean = x_train.mean()
#std = x_train.std()
#X_train = (x_train-mean)/std
#X_valid = (x_valid-mean)/std
#X_test = (x_test-mean)/std

In [ ]:
scaler = StandardScaler().fit(x_train)
X_train = scaler.transform(x_train)
X_valid = scaler.transform(x_valid)
X_test = scaler.transform(x_test)

# 建立模型



In [ ]:
filepath="./bestModel.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=False)
es = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='min')

In [ ]:
#initialRamdom = keras.initializers.random_normal(mean=0.0,stddev=0.05,seed=52)
model = keras.Sequential()
model.add(layers.Dense(64,kernel_initializer='normal',activation='relu',input_shape=(X_train.shape[1],)))
model.add(layers.Dense(48,kernel_initializer='normal',activation='relu'))
model.add(layers.Dense(32,kernel_initializer='normal',activation='relu'))
model.add(layers.Dense(1,kernel_initializer='normal'))
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(0.01978),loss='MAE')

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=64,epochs=1000,callbacks=[es,checkpoint],verbose=0)

In [ ]:
history.history.keys()

dict_keys(['loss', 'val_loss'])

In [ ]:
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='validation')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')

In [ ]:
from keras.models import load_model
filepath = "./bestModel.h5"
model = load_model(filepath)

In [ ]:
final_result=model.predict(X_test)

In [ ]:
final_result=np.reshape(final_result,(len(test_Data),))
print(final_result)

In [ ]:
with open('submit_results.csv', 'w') as f:
    f.write('Id,price\n')
    for i in range(len(final_result)):
        f.write(str(i+1) + ',' + str(final_result[i]) + '\n')